In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://www.aljazeera.com/where/mozambique/"
def getHTMLdocument(url):
    response = requests.get(url)
    return response.text

aljazeera_paper = getHTMLdocument(url)

In [4]:
base_url = "https://www.aljazeera.com"
soup = BeautifulSoup(aljazeera_paper, 'html.parser')
all_articles = soup.find_all("a", {"class": "u-clickable-card__link"})

article_links = [base_url + article['href'] for article in all_articles]

In [7]:
from newspaper import Article
import json
from tqdm import tqdm

article_collections = []

# article_url = article_links[0]
# article = Article(article_url)
# article.download()
# article.parse()
# article_soup = BeautifulSoup(article.html, 'html.parser')
# article_soup.find("p", {"class": "article__subhead"}).find('em').string

for article_url in tqdm(article_links):
    article = Article(article_url)
    article.download()
    article.parse()
    article_soup = BeautifulSoup(article.html, 'html.parser')
    subtitle = ''
    try:
        subtitle =  article_soup.find("p", {"class": "article__subhead"}).find('em').string
    except:
        pass
    article_collections.append(
        { 'title': article.title, 'subtitle': subtitle, 'content': article.text }
    )

# print(len(article_collections))
with open('data.json', 'w') as f:
    json.dump(article_collections, f)

# json.dumps(article_collections)

100%|██████████| 14/14 [00:04<00:00,  3.32it/s]


In [8]:
import nltk

nltk.download([
    "names",
    "stopwords",
    "punkt"
])

[nltk_data] Downloading package names to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
import json
from sentiment_analysis_spanish import sentiment_analysis

sentiment = sentiment_analysis.SentimentAnalysisSpanish()

f = open('data.json')

data = json.load(f)

text = data['content']

sentimentScore = sentiment.sentiment(text)
print(sentimentScore)

/Users/rhythmgirdhar/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/rhythmgirdhar/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


TypeError: list indices must be integers or slices, not str